## KeplerMapper-例3：同心円（ノイズあり②）

kmapper.KeplerMapperモジュールの解説：
https://kepler-mapper.scikit-tda.org/en/latest/reference/stubs/kmapper.KeplerMapper.html

パッケージのインストール

In [1]:
import kmapper as km 
from kmapper.plotlyviz import plotlyviz
import sklearn

KeplerMapperを用意

In [2]:
mapper = km.KeplerMapper(verbose=0) # verboseの引数は0（ログを取らない）か1（一部ログを取る）か2（完全にログを取る）．

データセットを用意

円周上に等間隔で分布するデータセット（データ点の数：5000，外側の円の半径：1，内側の円の半径：0.4，ガウシアンノイズ：標準偏差0.1）

In [3]:
from sklearn.datasets import make_circles

data, labels = make_circles(n_samples=5000, noise=0.1, factor=0.4, random_state=1)
data
# dataにはデータ点の座標の配列，labelsには各データ点に対応する正解ラベルの配列を代入．
# n_samples：データ点の数，noise：ガウシアンノイズの標準偏差（平均は0），factor：内側の円の半径．

array([[ 0.36780814,  0.20984398],
       [ 0.25914965, -0.16864038],
       [-0.33959456, -0.24145392],
       ...,
       [-0.49862473,  0.4924471 ],
       [-0.47811915, -0.11502908],
       [ 0.79721143,  0.51673327]])

In [4]:
# データセットの分布．
import numpy as np
import matplotlib.pyplot as plt
 
X0 = data[labels==0]
X1 = data[labels==1]
 
fig, ax = plt.subplots()
ax.scatter(X0[:, 0], X0[:, 1])
ax.scatter(X1[:, 0], X1[:, 1])
ax.axis('equal')
 
plt.show()

フィルター関数を考え，レンズ（フィルター関数によるデータ点の像の配列）を計算

フィルター関数：1つの軸（x軸）への写像：f:X（データセット）⊂R^2→R

レンズ：データ点のx座標

スケーリング：なし

In [5]:
lens = mapper.fit_transform(data, projection=[0], scaler=None) 
# projection：射影（デフォルト：sum），scaler：スケーリング（デフォルト：最小最大正規化）．
lens

array([[ 0.36780814],
       [ 0.25914965],
       [-0.33959456],
       ...,
       [-0.49862473],
       [-0.47811915],
       [ 0.79721143]])

被覆やクラスタリング手法を考え，Mapperグラフをつくる

n_cubes：20，percent overlap：10%

クラスタリング：DBSCAN（隣接点とみなす2点間の最大距離：0.1，ボーダー点の最小個数：10）

In [6]:
simplicial_complex = mapper.map(lens, X=data,
                                cover=km.Cover(n_cubes=20, perc_overlap=0.1),
                                clusterer=sklearn.cluster.DBSCAN(eps=0.1, min_samples=10))
# n_cubes：各軸の被覆の数
# perc_overlap：percent overlap，各被覆の長さに対する，隣の被覆と重なっている部分の長さの割合
# clusterer：クラスタリング手法

Mapperグラフを出力

色：レンズの最小値との距離

In [7]:
plotlyviz(simplicial_complex, title="Circle_noise2_v1", color_values=lens, dashboard=True) 
# cube番号：各クラスタが属する被覆の番号．被覆に含まれるデータ点が0個の場合は，設定したcube数の番号が全て含まれないこともある．
# cluster size：各クラスタに属するデータ点の数（members：データ点の番号）
# total smaples：各被覆に属するデータ点の数の総和（被覆が重なっていると重複カウントされる）
# （続き）percent overlapを大きくすると元のデータ点の数より多くなるが，被覆しきれていないと元のデータ点の数より少なくなる．
# unique samples：元のデータ点の数より少ない場合は外れ値を除去して数えている．

    'data': [{'hoverinfo': 'none',
              'line': {'color'…

In [8]:
_ = mapper.visualize(
        simplicial_complex,
        title='Circle_noise2_v1',
        path_html='Circle_noise2_v1.html',
        color_values=lens,
        color_function_name="lens",
    )